In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cdsapi
from mpetools import IslandTime
import pandas as pd
import datetime

In [2]:
cds = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key="200721:d13b27b3-32f8-4315-a9c0-e65dc3eb6fdd")

cds.retrieve(
    'reanalysis-era5-land-monthly-means',
    {
        'product_type': 'monthly_averaged_reanalysis',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
            '2m_temperature', 'evaporation_from_bare_soil', 'evaporation_from_open_water_surfaces_excluding_oceans',
            'evaporation_from_the_top_of_canopy', 'evaporation_from_vegetation_transpiration', 'leaf_area_index_high_vegetation',
            'leaf_area_index_low_vegetation', 'potential_evaporation', 'runoff',
            'skin_temperature', 'snow_evaporation', 'soil_temperature_level_1',
            'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4',
            'sub_surface_runoff', 'surface_pressure', 'surface_runoff',
            'total_evaporation', 'total_precipitation',
        ],
        'year': [
            '2010', '2011', '2012',
            '2013', '2014', '2015',
            '2016', '2017', '2018',
            '2019', '2020', '2021',
            '2022', '2023',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'format': 'netcdf.zip',
    },
    'download.netcdf.zip')

2023-09-26 10:18:58,112 INFO Welcome to the CDS
2023-09-26 10:18:58,112 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land-monthly-means
2023-09-26 10:18:58,729 INFO Request is completed
2023-09-26 10:18:58,730 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data1/adaptor.mars.internal-1695677232.6110673-20012-18-6ba50b98-0718-493b-9610-cf47a58e848b.zip to download.netcdf.zip (10.1G)
2023-09-26 10:38:57,758 INFO Download rate 8.6M/s     


Result(content_length=10807473857,content_type=application/zip,location=https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data1/adaptor.mars.internal-1695677232.6110673-20012-18-6ba50b98-0718-493b-9610-cf47a58e848b.zip)

In [5]:
island_info = IslandTime.retrieve_island_info('Dhiyadhoo', 'Maldives')


-------------------------------------------------------------------
Retrieving all information available for the island.
Island: Dhiyadhoo, Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               part of
               located in the administrative territorial entity
               located in or next to body of water
               atoll
spatial_reference
               latitude
               longitude
               polygon
               polygon_OSM
               transects_ECU_characteristics
               reference_shoreline
               transects
image_collection_dict
               description
               S2
               L7
               L8
               L9
timeseries_vegetation
               description
               description_timeseries
               source
               mask_total_vegetation_S2
               mask_coastal

In [6]:
island_info['timeseries_ERA5']['timeseries']['mean_2m_air_temperature'].plot()


<Axes: xlabel='datetime'>

In [7]:
polygon = island_info['spatial_reference']['polygon'].getInfo()['coordinates'][0]
area = [max([polygon[0][1], polygon[2][1]]), min([polygon[0][0], polygon[2][0]])+180, min([polygon[0][1], polygon[2][1]]), max([polygon[0][0], polygon[2][0]])+180]

In [8]:
import cdsapi
import xarray as xr
import numpy as np
import calendar

# Define the ERA5 data request parameters for daily data
start_year = 2017
end_year = 2019
variables = ['2m_temperature']

# Create an empty list to store the daily data
daily_data = []

cds = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key="200721:d13b27b3-32f8-4315-a9c0-e65dc3eb6fdd")

# Loop through each year and retrieve daily data
for year in range(start_year, end_year + 1):
    for variable in variables:
        era5_request_daily = {
            'product_type': 'reanalysis',
            'variable': variable,
            'year': str(year),
            'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
            'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                    '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
            'area': area,  # North, West, South, East
            'format': 'netcdf'
        }

        # Request daily ERA5 data
        file_name = f'era5_daily_{variable}_{year}.nc'
        cds.retrieve('reanalysis-era5-single-levels', era5_request_daily, file_name)

        # Load the data into an xarray dataset
        data_daily = xr.open_dataset(file_name)

        # Append to the daily data list
        daily_data.append(data_daily)

# Concatenate the daily data along the time dimension
combined_data = xr.concat(daily_data, dim='time')

# Print the combined dataset to view the data structure
print(combined_data)

# Save the combined data to a new NetCDF file
combined_data.to_netcdf('era5_daily_combined_temperature.nc')



2023-09-25 20:25:45,986 INFO Welcome to the CDS
2023-09-25 20:25:45,987 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-09-25 20:25:46,052 INFO Request is queued
2023-09-25 20:25:47,091 INFO Request is running
2023-09-25 20:26:18,486 INFO Request is completed
2023-09-25 20:26:18,486 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data3/adaptor.mars.internal-1695669966.299231-19784-11-c4702648-70ef-448f-a653-1418634df80d.nc to era5_daily_2m_temperature_2017.nc (3.2K)
2023-09-25 20:26:19,110 INFO Download rate 5.1K/s
2023-09-25 20:26:19,256 INFO Welcome to the CDS
2023-09-25 20:26:19,256 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-09-25 20:26:19,299 INFO Request is queued
2023-09-25 20:26:20,337 INFO Request is running
2023-09-25 20:26:51,729 INFO Request is completed
2023-09-25 20:26:51,730 INFO Downloading https://do

<xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 1095)
Coordinates:
  * longitude  (longitude) float32 253.5
  * latitude   (latitude) float32 0.468
  * time       (time) datetime64[ns] 2017-01-01T12:00:00 ... 2019-12-31T12:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 295.8 295.6 ... 296.4 296.4
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-25 19:26:07 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...


In [9]:
# Load the combined NetCDF file
file_path = 'era5_daily_combined_temperature.nc'
combined_data = xr.open_dataset(file_path)

# Extract the temperature data (assuming 't2m' is the variable for 2m temperature)
temperature_data = combined_data['t2m']

# Extract the time coordinates
time = combined_data['time']

# Convert time to a more human-readable format (e.g., as pandas datetime)
time_pd = pd.to_datetime(time.values)

In [13]:
plt.plot(time_pd, temperature_data[:, 0, 0], label='Daily Temperature')

In [67]:
plt.plot(df_varr.index, monthly_averages.variables['t2m'][:].values.flatten(), color='red')

In [12]:
varr = island_info['timeseries_ERA5']['timeseries']['mean_2m_air_temperature']
df_varr = varr[(varr.index >= datetime.datetime(year=2017, month=1, day=1)) & (varr.index <= datetime.datetime(year=2019, month=1, day=1))]
df_varr.plot()

<Axes: xlabel='datetime'>